In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
import joblib
warnings.filterwarnings('ignore')

# Загрузка данных
df = pd.read_csv('diamonds_train.csv')

# 1.ПРЕДОБРАБОТКА ДАННЫХ
# создание новых признаков
df['volume'] = df['x'] * df['y'] * df['z']
df['density'] = df['carat'] / (df['volume'] + 1e-8)  # избегаем деления на 0
df['table_ratio'] = df['table'] / (df['depth'] + 1e-8)

# кодирование категориальных переменных
cut_order = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}
color_order = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6}
clarity_order = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}

df['cut_encoded'] = df['cut'].map(cut_order)
df['color_encoded'] = df['color'].map(color_order)
df['clarity_encoded'] = df['clarity'].map(clarity_order)

# Убираем бесконечные значения
df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

# 2. ПОДГОТОВКА ПРИЗНАКОВ
base_features = ['carat', 'depth', 'table', 'x', 'y', 'z',
                'cut_encoded', 'color_encoded', 'clarity_encoded']
new_features = ['volume', 'density', 'table_ratio']
all_features = base_features + new_features

X = df[all_features]
y = df['price']

print(f"Признаки: {len(all_features)}")

# 3. РАЗДЕЛЕНИЕ ДАННЫХ
print("2. Разделение данных...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. МАСШТАБИРОВАНИЕ (только StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. ОЦЕНКА МОДЕЛЕЙ
# Используем только самые эффективные модели
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=10.0),  # увеличенный alpha для стабильности
    'Lasso Regression': Lasso(alpha=0.01, max_iter=1000),
    'Random Forest': RandomForestRegressor(
        n_estimators=50,  # уменьшено для скорости
        max_depth=15,
        random_state=42,
        n_jobs=-1  # используем все ядра
    )
}

results = []

for name, model in models.items():
    print(f"  Обучение {name}...")

    # Быстрое обучение
    model.fit(X_train_scaled, y_train)

    # Быстрые предсказания
    y_test_pred = model.predict(X_test_scaled)

    # Только основные метрики
    test_r2 = r2_score(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # Быстрая кросс-валидация (3-fold вместо 5)
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=3, scoring='r2')

    results.append({
        'Model': name,
        'Test R²': test_r2,
        'Test RMSE': test_rmse,
        'CV R² Mean': cv_scores.mean()
    })

# Анализ результатов
results_df = pd.DataFrame(results).sort_values('Test R²', ascending=False)
print("\nРЕЗУЛЬТАТЫ:")
print(results_df.round(4))

# 6. ФИНАЛЬНАЯ МОДЕЛЬ
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

print(f"\n4. Лучшая модель: {best_model_name}")

# 7. ФИНАЛЬНОЕ ОБУЧЕНИЕ НА ВСЕХ ДАННЫХ
print("5. Финальное обучение...")
X_scaled = scaler.fit_transform(X)
final_model = best_model
final_model.fit(X_scaled, y)

# оценка
y_pred = final_model.predict(X_scaled)
final_r2 = r2_score(y, y_pred)
final_rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"Финальные метрики:")
print(f"R²: {final_r2:.4f}")
print(f"RMSE: ${final_rmse:,.2f}")

# 8. АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ
if hasattr(final_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': all_features,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nТоп-5 важных признаков:")
    for i, row in feature_importance.head().iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")

# 9. СОХРАНЕНИЕ МОДЕЛИ
joblib.dump(final_model, 'fast_diamond_model.pkl')
joblib.dump(scaler, 'fast_scaler.pkl')

encoding_info = {
    'cut_order': cut_order,
    'color_order': color_order,
    'clarity_order': clarity_order
}
joblib.dump(encoding_info, 'fast_encoding_info.pkl')

# 10. ПРЕДСКАЗАНИЕ НА ТЕСТОВЫХ ДАННЫХ
print("\n6. Подготовка submission...")

df_test = pd.read_csv('diamonds_test.csv')

# предобработка тестовых данных
df_test['volume'] = df_test['x'] * df_test['y'] * df_test['z']
df_test['density'] = df_test['carat'] / (df_test['volume'] + 1e-8)
df_test['table_ratio'] = df_test['table'] / (df_test['depth'] + 1e-8)

df_test['cut_encoded'] = df_test['cut'].map(cut_order)
df_test['color_encoded'] = df_test['color'].map(color_order)
df_test['clarity_encoded'] = df_test['clarity'].map(clarity_order)

df_test = df_test.replace([np.inf, -np.inf], np.nan).fillna(0)

# Предсказания
X_test_final = df_test[all_features]
X_test_scaled_final = scaler.transform(X_test_final)
test_predictions = final_model.predict(X_test_scaled_final)

# Сохранение
submission_df = pd.DataFrame({
    'id': df_test['id'],
    'price': test_predictions
})

submission_df['price'] = submission_df['price'].clip(lower=0)
submission_df.to_csv('submission.csv', index=False)

=== ОПТИМИЗИРОВАННАЯ МОДЕЛЬ ===
1. Быстрая предобработка...
Признаки: 12
2. Разделение данных...
3. Обучение моделей...
  Обучение Linear Regression...
  Обучение Ridge Regression...
  Обучение Lasso Regression...
  Обучение Random Forest...

РЕЗУЛЬТАТЫ:
               Model  Test R²  Test RMSE  CV R² Mean
3      Random Forest   0.9782   592.9679      0.9805
2   Lasso Regression   0.3583  3220.2893      0.9110
1   Ridge Regression   0.2627  3451.9060      0.9127
0  Linear Regression  -0.2460  4487.2974      0.9031

4. Лучшая модель: Random Forest
5. Финальное обучение...
Финальные метрики:
R²: 0.9949
RMSE: $284.92

Топ-5 важных признаков:
  volume: 0.6544
  y: 0.2082
  clarity_encoded: 0.0626
  color_encoded: 0.0313
  carat: 0.0278

6. Подготовка submission...
Готово! Файл: fast_submission.csv
Предсказано цен: 5379
Диапазон: $375.08 - $18078.98
